# 0. Importing data & cast_list_as_strings

In [1]:
import sklearn
from sklearn import *
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import xgboost as xgb
import pandas as pd
import numpy as np
import scipy
import os
import statistics
import editdistance
import itertools  
import re
import unicodedata
import string

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
cl = sklearn.linear_model.Perceptron()

In [7]:
path_data =  os.path.expanduser('~') 


# use this to train and VALIDATE your solution
train_df = pd.read_csv(os.path.join(path_data,
                                    os.path.join("Datasets", "kaggle_datasets", "quora", "quora_train_data.csv")))

# use this to provide the expected generalization results
test_df = pd.read_csv(os.path.join(path_data,
                                    os.path.join("Datasets", "kaggle_datasets", "quora", "test.csv"))) # "test.csv" is the same as "quora_test_data.csv"

In [8]:
train_df.loc[3306,'question1']

'.'

# 1. Raw data cleaning and preprocessing 

###  Exercise 1:  `cast_list_as_strings`

Build a function  **`cast_list_as_strings`** that casts each element in the input list to a string.


In [10]:
all_q = list(train_df["question1"]) + list(train_df["question2"])

In [11]:
types_ = [type(q).__name__ for q in all_q]
np.unique(types_)

array(['float', 'str'], dtype='<U5')

In [12]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

In [13]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
#q1_test  =  cast_list_as_strings(list(test_df["question1"]))
#q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [14]:
types_ = [type(q).__name__ for q in q2_train]
np.unique(types_)

array(['str'], dtype='<U3')

### Remove Punctuation, substituting and lowering
Stolen from https://www.kaggle.com/benjaminkz/quora-question-pairs-data-cleaning

In [15]:
def common_words_transformation_remove_punctuation(text):
    
    text = text.lower()
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"when's", "when is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"there's", "there is", text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)  # 除了上面的特殊情况外，“\'s”只能表示所有格，应替换成“ ”
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " america ", text)
    text = re.sub(r" u s ", " america ", text)
    text = re.sub(r" uk ", " england ", text)
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text)
    text = re.sub(r" ds ", " data science ", text)
    text = re.sub(r" ee ", " electronic engineering ", text)
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iphone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the us", "america", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    
    punctuation= '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'


    text = "".join([c for c in text if c not in punctuation])
    
    return text

In [16]:
doc = train_df['question1'][0]
doc

'What is the step by step guide to invest in share market in india?'

In [17]:
doc = common_words_transformation_remove_punctuation(doc)
doc

'what is the step by step guide to invest in share market in india '

### Removing accents

In [18]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

### Removing Stop_Words

In [20]:
def remove_stopwords(doc):
    
    stopwords_list = set(stopwords.words('english'))
    stops = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
    all_stopwords = (stopwords_list + list(set(stops) - set(stopwords_list)))
    token_doc = set(word_tokenize(doc))
    token_doc = list(token_doc - all_stopwords)
    return " ".join(token_doc)

### Stemming

In [22]:
porter    = PorterStemmer()
#lancaster = LancasterStemmer()
#Snowball = SnowballStemmer(language='english')

In [23]:
def stem(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(porter.stem(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

### Lemmatization

In [25]:
#nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

In [26]:
def lemm(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(wordnet_lemmatizer.lemmatize(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

### Putting all together

In [30]:
#this function is creating different versions of the questions based on the different cleaning methods defined above.
#once we understand which of these methods work best, we will combined them in question1_preprocessed and question2_preprocessed.

def preprocess(df, save=False):
    df = df.copy()
    df['question1'] = cast_list_as_strings(list(df["question1"]))
    df['question2'] = cast_list_as_strings(list(df["question2"]))
        
    df['question1_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question1"]]
    df['question2_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question2"]]
    
    df['question1_noAccents'] = [remove_accented_chars(quest) for quest in df["question1"]]
    df['question2_noAccents'] = [remove_accented_chars(quest) for quest in df["question2"]]
    
    df['question1_no_stops'] = [remove_stopwords(quest) for quest in df["question1_cleaned"]]
    df['question2_no_stops'] = [remove_stopwords(quest) for quest in df["question2_cleaned"]]
    
    df['question1_Porter_stemmed'] = [stem(quest) for quest in df["question1_no_stops"]]
    df['question2_Porter_stemmed'] = [stem(quest) for quest in df["question2_no_stops"]]
        
    df['question1_Lemmatization']  = [lemm(quest) for quest in df["question1_no_stops"]]
    df['question2_Lemmatization']  = [lemm(quest) for quest in df["question2_no_stops"]]
    
    
    if save==True:
        df.to_csv('preprocessed_Quora_full_train_data.csv')
        
    return df

In [32]:
full_train_proc = preprocess(train_df, save=False)

TypeError: unsupported operand type(s) for +: 'set' and 'list'

In [ ]:
full_train_proc.head()

## Adjusting empty questions

In [ ]:
def cont_len(question):
    num_words = len(question.split())
    return num_words

#Dropping all the sample where there is not one of the two questions
len_1   = full_train_proc.question1_cleaned.apply(lambda x: cont_len(x))
empty_1 = len_1[len_1==0].index

len_2   = full_train_proc.question2_cleaned.apply(lambda x: cont_len(x))
empty_2 = len_2[len_2==0].index

to_delete = (empty_2).union(set(empty_1))

full_train_proc = full_train_proc.drop(index=to_delete)

## Using CountVectorizer Solution
### Naive CountVectorizer and attempts to improve it


Use all the questions in train and test partitions to build a single list `all_questions` to fit the `count_vectorizer`

In [20]:
all_questions = q1_train + q2_train

In [22]:
df = train_df.copy()
df['question1_string'] = cast_list_as_strings(list(df["question1"]))
df['question2_string'] = cast_list_as_strings(list(df["question2"]))

In [82]:
count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer='word', binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, tokenizer=None, vocabulary=None)

count_vectorizer.fit(all_questions)

CountVectorizer(stop_words='english', strip_accents='unicode')

In [74]:
#advanced version with stemming
#this implemenetation failed because stop_words cannot be used if analyzer!='word'

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

lemmatizer = WordNetLemmatizer()
stemmer =  SnowballStemmer(language='english')
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer=stemmed_words, binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, tokenizer=None, vocabulary=None)

count_vectorizer.fit(all_questions)

/Users/giuliocaggiano/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:497: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


CountVectorizer(analyzer=<function stemmed_words at 0x7f86a4e875e0>,
                stop_words='english', strip_accents='unicode')

In [78]:
#advanced version with lemmatization on top of stemming
#this implemenetation failed because the tokenizer cannot be used if analyzer!='word'
#additionally, if tokenizer!=None token_pattern and stop_words cannot be used

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

lemmatizer = WordNetLemmatizer()
stemmer =  SnowballStemmer(language='english')
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer=stemmed_words, binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, 
                                                                 tokenizer=LemmaTokenizer(), 
                                                                 vocabulary=None)

count_vectorizer.fit(all_questions)

/Users/giuliocaggiano/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/Users/giuliocaggiano/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:504: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'tokenizer' will not be used"


CountVectorizer(analyzer=<function stemmed_words at 0x7f86a4e87700>,
                stop_words='english', strip_accents='unicode',
                tokenizer=<__main__.LemmaTokenizer object at 0x7f8710b9d100>)

#### Problems with the improvement attempts so far

* if analyzer=stemmed_words --> stop_words and tokenizer cannot be used
* if tokenizer=LemmaTokenizer() --> token_pattern and stop_words cannot be used

To solve these issues, we will not use the analyzer. We tokenize, remove the punctuation and stopwords, apply the regEx, and stem all inside the tokenizer.

#### Final CountVectorizer!

In [2]:
stopwords_list=nltk.corpus.stopwords.words('english')
stops = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
stopwords_punctuation = (stopwords_list + list(set(stops) - set(stopwords_list)) + list(string.punctuation))

class Tokenizer(object):
    def __init__(self):
        self.stemmer = porter
        self.lemmatizer = wordnet_lemmatizer
    def __call__(self, articles):
        tokens = re.compile(r'(?u)\b\w\w+\b').findall(articles) 
        tokens = [' '.join(w for w in token.split() if w.lower() not in stopwords_punctuation) for token in tokens]
        tokens_stem = [self.stemmer.stem(t) for t in tokens]
        tokens_lem = [self.lemmatizer.lemmatize(t) for t in tokens_stem]
        return tokens_lem

In [12]:
count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer='word', binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 preprocessor=None, 
                                                                 tokenizer=Tokenizer(), 
                                                                 vocabulary=None)

count_vectorizer.fit(all_questions)

/Users/giuliocaggiano/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(strip_accents='unicode',
                tokenizer=<__main__.Tokenizer object at 0x7fe9f87e2e80>)

# 2. Other Feature Vectors

In [33]:
#Adding len Words in common feat
def len_common(q1, q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1.intersection(q2))

#Adding len common words in common feat
def len_not_common(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1 ^ q2)

#Adding mean distance between common words 
def mean_dist_not_com(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    not_comm1 = (q1 ^ q2) - q1
    if len(not_comm1)==0 : not_comm1={''}
    not_comm2 = (q1 ^ q2) - q2
    if len(not_comm2)==0 : not_comm2={''}
    return statistics.mean([editdistance.eval(i[0],i[1]) for i in itertools.product(not_comm1, not_comm2)])

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString)) 
def both_number(q1,q2):
    return hasNumbers(q1) *  hasNumbers(q2) 

# get average number of words
def average_len(question):
    num_words = len(question.split())
    return len(question)/num_words   

# levenshtein distance (for strings of unequal length)
def levenshtein(q1, q2): 
    #create initial array (two for loops since q1 and q2 can differ in length)
    dist_array = []
    for i in range(len(q1)+1):
        dist_array.append([0]*(len(q2)+1))
        dist_array[i][0] = i
    for j in range(len(q2)+1):
        dist_array[0][j] = j

    dist = [0]*3
    for i in range(1,len(q1)+1):
        for j in range(1,len(q2)+1):
            dist[0] = dist_array[i-1][j-1] if q1[i-1]==q2[j-1] else dist_array[i-1][j-1]+1
            dist[1] = dist_array[i][j-1]+1
            dist[2] = dist_array[i-1][j]+1
            dist_array[i][j]=min(dist)
    
    return dist_array[i][j]

# self implemented jaccard similarity
#from math import*
 
def jaccard_similarity(vector1,vector2):
    jacc_num = 0 
    jacc_den = 0 
    for index in enumerate(vector1): 
        if vector1[index] != 0 or vector2[index] != 0: 
            jacc_den += max(vector1[index], vector2[index]) 
            jacc_num += min(vector1[index], vector2[index]) 
    return jacc_num / jacc_den

def common_tokens(string_1,string_2):
    """
    counts common word types. options are NOUN,VERB,ADV,ADJ
    """
    
    string_1 = str(string_1)
    string_2 = str(string_2)
    
    x = nltk.word_tokenize(string_1)
    y = nltk.word_tokenize(string_2)   
 
    common_tokens = len(list(set(x).intersection(y)))
    
    return(common_tokens)

def common_count(string_1,string_2, word_type = "NOUN"):
    """
    counts common word types. options are NOUN,VERB,ADV,ADJ
    """
    string_1 = str(string_1)
    string_2 = str(string_2)
    
    tagged_1 = nltk.pos_tag(nltk.word_tokenize(string_1),tagset='universal')
    tagged_2 = nltk.pos_tag(nltk.word_tokenize(string_2),tagset='universal')
    
    x = list([])
    for word in tagged_1:
        if word[1] == word_type:
            x.append(word[0])
                     
    y = list([])
    for word in tagged_2:
        if word[1] == word_type:
            y.append(word[0])
 
    common = len(list(set(x).intersection(y)))
    
    return(common)

def length_diff_characters(x,y):
    """
    find absolute difference in character length of two questions
    """
    x = str(x)
    y = str(y)
    return abs(len(y)-len(x))

def length_diff_tokens(x,y):
    """
    find absolute difference in number of tokens in two questions
    """
    return abs(len(nltk.word_tokenize(y))-len(nltk.word_tokenize(x)))

def common_numbers(x,y):
    """
    count numbers present in both tokens
    """

    x_numbers = re.findall(r'\b\d+\b', x)
    y_numbers = re.findall(r'\b\d+\b', y)
    
    common_numbers = len(list(set(x_numbers).intersection(y_numbers)))
    return(common_numbers)




In [188]:
#nltk.download('averaged_perceptron_tagger')
#nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/pablogranatiero/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
#df_test["common_nouns"] = df_test.apply(lambda df_test: common_count(df_test.question1, df_test.question2,"NOUN"), axis=1)
#df_test["common_verbs"] = df_test.apply(lambda df_test: common_count(df_test.question1, df_test.question2,"VERB"), axis=1)
#df_test["common_adjectives"] = df_test.apply(lambda df_test: common_count(df_test.question1, df_test.question2,"ADJ"), axis=1)
#df_test["common_adverbs"] = df_test.apply(lambda df_test: common_count(df_test.question1, df_test.question2,"ADV"), axis=1)
#df_test["common_tokens"] = df_test.apply(lambda df_test: common_tokens(df_test.question1, df_test.question2), axis=1)
#df_test["length_diff_characters"] = df_test.apply(lambda df_test: length_diff_characters(df_test.question1, df_test.question2), axis=1)
#df_test["length_diff_tokens"] = df_test.apply(lambda df_test: length_diff_tokens(df_test.question1, df_test.question2), axis=1)
#df_test["common_numbers"] = df_test.apply(lambda df_test: length_diff_characters(df_test.question1, df_test.question2), axis=1)


In [163]:
def Add_features(df, scaler, col1, col2, save=False):
    df = df.copy()
    col1_to_transform = list(full_train_proc.columns).index(col1)
    col2_to_transform = list(full_train_proc.columns).index(col2)

    df['len_q1'] = [len(s) for s in df[col1]] 
    df['len_q2'] = [len(s) for s in df[col2]]
    df['len_q1'] = scaler.fit_transform(df[['len_q1']])
    df['len_q2'] = scaler.fit_transform(df[['len_q2']])
    
    df['len_common'] = [len_common(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['len_common'] = scaler.fit_transform(df[['len_common']])

    df['len_not_common'] = [len_not_common(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['len_not_common'] = scaler.fit_transform(df[['len_not_common']])
    
    df['mean_dist_not_com'] = [mean_dist_not_com(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['mean_dist_not_com'] = scaler.fit_transform(df[['mean_dist_not_com']])
    
    df['both_number'] = [both_number(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    
    df["avg_len_q1"]= df[col1].apply(lambda x: average_len(x))
    df["avg_len_q2"]= df[col2].apply(lambda x: average_len(x))
    df['avg_len_q1'] = scaler.fit_transform(df[['avg_len_q1']])
    df['avg_len_q2'] = scaler.fit_transform(df[['avg_len_q2']])

    df['edit_distance'] = [editdistance.eval(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['edit_distance'] = scaler.fit_transform(df[['edit_distance']])
    
    df['levenshtein_distance'] = [levenshtein(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['levenshtein_distance'] = scaler.fit_transform(df[['levenshtein_distance']])
    
    #df['manhattan_distance'] = [manhattan(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    #df['manhattan_distance'] = scaler.fit_transform(df[['manhattan_distance']])
    df["common_nouns"] = df.apply(lambda df: common_count(df[col1], df[col2],"NOUN"), axis=1)
    df['common_nouns'] = scaler.fit_transform(df[['common_nouns']])

    df["common_verbs"] = df.apply(lambda df: common_count(df[col1], df[col2],"VERB"), axis=1)
    df['common_verbs'] = scaler.fit_transform(df[['common_verbs']])
    
    df["common_adjectives"] = df.apply(lambda df: common_count(df[col1], df[col2],"ADJ"), axis=1)
    df['common_adjectives'] = scaler.fit_transform(df[['common_adjectives']])
    
    df["common_adverbs"] = df.apply(lambda df: common_count(df[col1], df[col2],"ADV"), axis=1)
    df['common_adverbs'] = scaler.fit_transform(df[['common_adverbs']])

    df["common_tokens"] = df.apply(lambda df: common_tokens(df[col1], df[col2]), axis=1)
    df['common_tokens'] = scaler.fit_transform(df[['common_tokens']])

    df["length_diff_characters"] = df.apply(lambda df: length_diff_characters(df[col1], df[col2]), axis=1)
    df['length_diff_characters'] = scaler.fit_transform(df[['length_diff_characters']])

    df["length_diff_tokens"] = df.apply(lambda df: length_diff_tokens(df[col1], df[col2]), axis=1)
    df['length_diff_tokens'] = scaler.fit_transform(df[['length_diff_tokens']])

    df["common_numbers"] = df.apply(lambda df: length_diff_characters(df.question1, df.question2), axis=1)
    df['common_numbers'] = scaler.fit_transform(df[['common_numbers']])

    if save==True:
        df.to_csv('features_added_Quora_full_train_data.csv')
    
    return df

In [175]:
len(full_train_proc_plus_feat)

404269

In [164]:
scaler = MinMaxScaler()

full_train_proc_plus_feat = Add_features(full_train_proc, scaler, 'question1_cleaned', 'question2_cleaned', save=True)


In [169]:
full_train_proc_plus_feat.to_csv('features_added_Quora_full_train_data.csv')

# 3. Two Simple Models

###  Exercise 2:  `get_features_from_df`

Make a function `get_features_from_df` that given a dataframe containing the format of the train data
it returns a scipy sparse matrix with the features from question 1 and question 2 

In [170]:
def fit_on_q1_q2(df, model):
    q_list1 = list(df["question1_cleaned"])
    q_list2 = list(df["question2_cleaned"])
    all_questions = q_list1 + q_list2 
    model.fit(all_questions)
    return

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    q2_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2

Training a Logistic and a XGB on the cleaned questions using BOW

In [197]:


def train_models(df, get_feat_model, vectorizer_func, col_list=0, add_feat=False):
    
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, 
                              learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
                              objective='binary:logistic', eta=0.3, silent=1, subsample=0.8, random_state=123)

    #svm_model = svm.SVC()
    
    train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_feat_model(train_df, vectorizer_func)
    X_val_q1q2        = get_feat_model(val_df, vectorizer_func)
    X_test_q1q2       = get_feat_model(test_df, vectorizer_func)
    
    if add_feat==True:
        X_train_q1q2      = add_a_column_feat_single(train_df, col_list, X_train_q1q2)
        X_val_q1q2        = add_a_column_feat_single(val_df, col_list, X_val_q1q2)
        X_test_q1q2       = add_a_column_feat_single(test_df, col_list, X_test_q1q2)
    
    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)
    xgb_model.fit(X_train_q1q2, y_train) 
    #svm_model.fit(X_tr_q1q2, y_train)
                                                   
    logistic_train_acc = roc_auc_score(y_train, logistic.predict_proba(X_train_q1q2)[:, 1])                                                       
    logistic_val_acc   = roc_auc_score(y_train, logistic.predict_proba(X_val_q1q2)[:, 1])
    logistic_test_acc  = roc_auc_score(y_train, logistic.predict_proba(X_test_q1q2)[:, 1])
    print('logistic_train_acc:{}, logistic_val_acc:{}, logistic_test_acc:{}'.format(logistic_train_acc, logistic_val_acc, logistic_test_acc))
                                                   
    xgb_train_acc      = roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])
    xgb_val_acc        = roc_auc_score(y_val, xgb_model.predict_proba(X_val_q1q2)[:, 1])
    xgb_test_acc       = roc_auc_score(y_test, xgb_model.predict_proba(X_test_q1q2)[:, 1])
    print('xgb_train_acc:{}, xgb_val_acc:{}, xgb_test_acc:{}'.format(xgb_train_acc, xgb_val_acc, xgb_test_acc))
                                                   
    return logistic, xgb_model, [logistic_train_acc, logistic_val_acc, logistic_test_acc], [xgb_train_acc, xgb_val_acc, xgb_test_acc]
                                              
                                                       
                                                       
           

In [177]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

fit_on_q1_q2(full_train_proc_plus_feat, count_vectorizer)

In [179]:
logistic_simple, xgb_model_simple, acc_logistic_simple, acc_xgboost_simple = train_models(full_train_proc_plus_feat, get_features_from_df, count_vectorizer)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:45:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:45:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8560015673032263, logistic_val_acc:0.7953934745032775, logistic_test_acc:0.7956985843493918
xgb_train_acc:0.8659191549450259, xgb_val_acc:0.8093737107889942, xgb_test_acc:0.8097783288425393


Training a Logistic and a XGB on the cleaned questions using tfidf

In [180]:
def get_tfidf(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    q2_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [181]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(full_train_proc_plus_feat, tfidf)

In [182]:
logistic_simple_tf, xgb_model_simple_tf, acc_logistic_simple_tf, acc_xgboost_simple_tf = train_models(full_train_proc_plus_feat, get_tfidf, tfidf)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:58:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8416865189699401, logistic_val_acc:0.7958238445455491, logistic_test_acc:0.797239576925177
xgb_train_acc:0.9070173792036841, xgb_val_acc:0.8234103363606295, xgb_test_acc:0.827242512987826


# 3. More complex models - Adding Features

### Adding cosine similarity

In [190]:
def get_tfidf(df, tfidf, sim=True):
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 
    
    

In [191]:
logistic_simple_tf_sim, xgb_model_simple_tf_sim, acc_logistic_tf_sim, acc_xgboost_tf_sim = train_models(full_train_proc_plus_feat, get_tfidf, tfidf)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:56:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:56:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.9074969597126898, logistic_val_acc:0.8726778599886169, logistic_test_acc:0.8772466341269656
xgb_train_acc:0.9766716766767485, xgb_val_acc:0.8987541590750521, xgb_test_acc:0.9019259149390173


### Adding features based on lenght

In [193]:
def add_a_column_feat(df, col_list, sparse_matrix):

    for col_q in col_list:
        feat_q = df[col_q].to_numpy().reshape(len(df[col_q]),1)
        sparse_matrix = scipy.sparse.hstack((sparse_matrix,feat_q)).tocsr()
    
    return sparse_matrix     

In [ ]:
cols_lenght = ['len_q1', 'len_q2', 'len_common', 'len_not_common']
logistic_simple_tf_sim_len, xgb_model_simple_tf_sim_len, acc_logistic_tfidf_sim_len, acc_xgboost_tfidf_sim_len = train_models(full_train_proc_plus_feat, get_tfidf, tfidf, [cols_lenght], add_feat=True)     
                                                                                                                              
